In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
from datetime import datetime
import hashlib
import os.path
import random
import re
import sys
import tarfile
import time

import numpy as np
from six.moves import urllib
import tensorflow as tf

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat

In [71]:
intermediate_store_frequency=0
learning_rate=0.0005
testing_percentage=10
validation_percentage=10
eval_step_interval=10
train_batch_size=50
test_batch_size=-1
validation_batch_size=50
print_misclassified_test_images=False
final_tensor_name='final_result'
architecture='mobilenet_1.0_224'
how_many_training_steps=5000
model_dir='tf_files/models/'
bottleneck_dir='tf_files/bottlenecks'
output_graph='tf_files/retrained_graph.pb'
output_labels='tf_files/retrained_labels.txt'
image_dir='tf_files/YE358311_Fender_apron'
summaries_dir='tf_files/training_summaries/Mobilenet_0.0005_5000'
intermediate_output_graphs_dir='/tmp/intermediate_graph/'
MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1

In [47]:
# model_info={
#       'data_url': 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz',
#       'bottleneck_tensor_name': 'pool_3/_reshape:0',
#       'bottleneck_tensor_size': 2048,
#       'input_width': 299,
#       'input_height': 299,
#       'input_depth': 3,
#       'resized_input_tensor_name': 'Mul:0',
#       'model_file_name': 'classify_image_graph_def.pb',
#       'input_mean': 128,
#       'input_std': 128,
#   }
model_info= {'data_url': 'http://download.tensorflow.org/models/mobilenet_v1_1.0_224_frozen.tgz',
 'bottleneck_tensor_name': 'MobilenetV1/Predictions/Reshape:0',
 'bottleneck_tensor_size': 1001,
 'input_width': 224,
 'input_height': 224,
 'input_depth': 3,
 'resized_input_tensor_name': 'input:0',
 'model_file_name': 'mobilenet_v1_1.0_224\\frozen_graph.pb',
 'input_mean': 127.5,
 'input_std': 127.5}

In [48]:
def create_image_lists(image_dir, testing_percentage, validation_percentage):
  """Builds a list of training images from the file system.

  Analyzes the sub folders in the image directory, splits them into stable
  training, testing, and validation sets, and returns a data structure
  describing the lists of images for each label and their paths.

  Args:
    image_dir: String path to a folder containing subfolders of images.
    testing_percentage: Integer percentage of the images to reserve for tests.
    validation_percentage: Integer percentage of images reserved for validation.

  Returns:
    A dictionary containing an entry for each label subfolder, with images split
    into training, testing, and validation sets within each label.
  """
  if not gfile.Exists(image_dir):
    tf.logging.error("Image directory '" + image_dir + "' not found.")
    return None
  result = collections.OrderedDict()
  sub_dirs = [
    os.path.join(image_dir,item)
    for item in gfile.ListDirectory(image_dir)]
  sub_dirs = sorted(item for item in sub_dirs
                    if gfile.IsDirectory(item))
  for sub_dir in sub_dirs:
    extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
    file_list = []
    dir_name = os.path.basename(sub_dir)
    if dir_name == image_dir:
      continue
    tf.logging.info("Looking for images in '" + dir_name + "'")
    for extension in extensions:
      file_glob = os.path.join(image_dir, dir_name, '*.' + extension)
      file_list.extend(gfile.Glob(file_glob))
    if not file_list:
      tf.logging.warning('No files found')
      continue
    if len(file_list) < 20:
      tf.logging.warning(
          'WARNING: Folder has less than 20 images, which may cause issues.')
    elif len(file_list) > MAX_NUM_IMAGES_PER_CLASS:
      tf.logging.warning(
          'WARNING: Folder {} has more than {} images. Some images will '
          'never be selected.'.format(dir_name, MAX_NUM_IMAGES_PER_CLASS))
    label_name = re.sub(r'[^a-z0-9]+', ' ', dir_name.lower())
    training_images = []
    testing_images = []
    validation_images = []
    for file_name in file_list:
      base_name = os.path.basename(file_name)
      # We want to ignore anything after '_nohash_' in the file name when
      # deciding which set to put an image in, the data set creator has a way of
      # grouping photos that are close variations of each other. For example
      # this is used in the plant disease data set to group multiple pictures of
      # the same leaf.
      hash_name = re.sub(r'_nohash_.*$', '', file_name)
      # This looks a bit magical, but we need to decide whether this file should
      # go into the training, testing, or validation sets, and we want to keep
      # existing files in the same set even if more files are subsequently
      # added.
      # To do that, we need a stable way of deciding based on just the file name
      # itself, so we do a hash of that and then use that to generate a
      # probability value that we use to assign it.
      hash_name_hashed = hashlib.sha1(compat.as_bytes(hash_name)).hexdigest()
      percentage_hash = ((int(hash_name_hashed, 16) %
                          (MAX_NUM_IMAGES_PER_CLASS + 1)) *
                         (100.0 / MAX_NUM_IMAGES_PER_CLASS))
      if percentage_hash < validation_percentage:
        validation_images.append(base_name)
      elif percentage_hash < (testing_percentage + validation_percentage):
        testing_images.append(base_name)
      else:
        training_images.append(base_name)
    result[label_name] = {
        'dir': dir_name,
        'training': training_images,
        'testing': testing_images,
        'validation': validation_images,
    }
  return result

In [49]:
def get_image_path(image_lists, label_name, index, image_dir, category):
  """"Returns a path to an image for a label at the given index.

  Args:
    image_lists: Dictionary of training images for each label.
    label_name: Label string we want to get an image for.
    index: Int offset of the image we want. This will be moduloed by the
    available number of images for the label, so it can be arbitrarily large.
    image_dir: Root folder string of the subfolders containing the training
    images.
    category: Name string of set to pull images from - training, testing, or
    validation.

  Returns:
    File system path string to an image that meets the requested parameters.

  """
  if label_name not in image_lists:
    tf.logging.fatal('Label does not exist %s.', label_name)
  label_lists = image_lists[label_name]
  if category not in label_lists:
    tf.logging.fatal('Category does not exist %s.', category)
  category_list = label_lists[category]
  if not category_list:
    tf.logging.fatal('Label %s has no images in the category %s.',
                     label_name, category)
  mod_index = index % len(category_list)
  base_name = category_list[mod_index]
  sub_dir = label_lists['dir']
  full_path = os.path.join(image_dir, sub_dir, base_name)
  return full_path


In [50]:
def get_bottleneck_path(image_lists, label_name, index, bottleneck_dir,
                        category, architecture):
  """"Returns a path to a bottleneck file for a label at the given index.

  Args:
    image_lists: Dictionary of training images for each label.
    label_name: Label string we want to get an image for.
    index: Integer offset of the image we want. This will be moduloed by the
    available number of images for the label, so it can be arbitrarily large.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    category: Name string of set to pull images from - training, testing, or
    validation.
    architecture: The name of the model architecture.

  Returns:
    File system path string to an image that meets the requested parameters.
  """
  return get_image_path(image_lists, label_name, index, bottleneck_dir,
                        category) + '_' + architecture + '.txt'

In [51]:
def create_model_graph(model_info):
  """"Creates a graph from saved GraphDef file and returns a Graph object.

  Args:
    model_info: Dictionary containing information about the model architecture.

  Returns:
    Graph holding the trained Inception network, and various tensors we'll be
    manipulating.
  """
  with tf.Graph().as_default() as graph:
    model_path = os.path.join(model_dir, model_info['model_file_name'])
    with gfile.FastGFile(model_path, 'rb') as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
      bottleneck_tensor, resized_input_tensor = (tf.import_graph_def(
          graph_def,
          name='',
          return_elements=[
              model_info['bottleneck_tensor_name'],
              model_info['resized_input_tensor_name'],
          ]))
  return graph, bottleneck_tensor, resized_input_tensor


In [52]:
def run_bottleneck_on_image(sess, image_data, image_data_tensor,
                            decoded_image_tensor, resized_input_tensor,
                            bottleneck_tensor):
  """Runs inference on an image to extract the 'bottleneck' summary layer.

  Args:
    sess: Current active TensorFlow Session.
    image_data: String of raw JPEG data.
    image_data_tensor: Input data layer in the graph.
    decoded_image_tensor: Output of initial image resizing and  preprocessing.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: Layer before the final softmax.

  Returns:
    Numpy array of bottleneck values.
  """
  # First decode the JPEG image, resize it, and rescale the pixel values.
  resized_input_values = sess.run(decoded_image_tensor,
                                  {image_data_tensor: image_data})
  # Then run it through the recognition network.
  bottleneck_values = sess.run(bottleneck_tensor,
                               {resized_input_tensor: resized_input_values})
  bottleneck_values = np.squeeze(bottleneck_values)
  return bottleneck_values

In [53]:
def ensure_dir_exists(dir_name):
  """Makes sure the folder exists on disk.

  Args:
    dir_name: Path string to the folder we want to create.
  """
  if not os.path.exists(dir_name):
    os.makedirs(dir_name)


In [54]:
def create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor):
  """Create a single bottleneck file."""
  tf.logging.info('Creating bottleneck at ' + bottleneck_path)
  image_path = get_image_path(image_lists, label_name, index,
                              image_dir, category)
  if not gfile.Exists(image_path):
    tf.logging.fatal('File does not exist %s', image_path)
  image_data = gfile.FastGFile(image_path, 'rb').read()
  try:
    bottleneck_values = run_bottleneck_on_image(
        sess, image_data, jpeg_data_tensor, decoded_image_tensor,
        resized_input_tensor, bottleneck_tensor)
  except Exception as e:
    raise RuntimeError('Error during processing file %s (%s)' % (image_path,
                                                                 str(e)))
  bottleneck_string = ','.join(str(x) for x in bottleneck_values)
  with open(bottleneck_path, 'w') as bottleneck_file:
    bottleneck_file.write(bottleneck_string)

In [55]:
def get_or_create_bottleneck(sess, image_lists, label_name, index, image_dir,
                             category, bottleneck_dir, jpeg_data_tensor,
                             decoded_image_tensor, resized_input_tensor,
                             bottleneck_tensor, architecture):
  """Retrieves or calculates bottleneck values for an image.

  If a cached version of the bottleneck data exists on-disk, return that,
  otherwise calculate the data and save it to disk for future use.

  Args:
    sess: The current active TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    label_name: Label string we want to get an image for.
    index: Integer offset of the image we want. This will be modulo-ed by the
    available number of images for the label, so it can be arbitrarily large.
    image_dir: Root folder string  of the subfolders containing the training
    images.
    category: Name string of which  set to pull images from - training, testing,
    or validation.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    jpeg_data_tensor: The tensor to feed loaded jpeg data into.
    decoded_image_tensor: The output of decoding and resizing the image.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The output tensor for the bottleneck values.
    architecture: The name of the model architecture.

  Returns:
    Numpy array of values produced by the bottleneck layer for the image.
  """
  label_lists = image_lists[label_name]
  sub_dir = label_lists['dir']
  sub_dir_path = os.path.join(bottleneck_dir, sub_dir)
  ensure_dir_exists(sub_dir_path)
  bottleneck_path = get_bottleneck_path(image_lists, label_name, index,
                                        bottleneck_dir, category, architecture)
  if not os.path.exists(bottleneck_path):
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor)
  with open(bottleneck_path, 'r') as bottleneck_file:
    bottleneck_string = bottleneck_file.read()
  did_hit_error = False
  try:
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  except ValueError:
    tf.logging.warning('Invalid float found, recreating bottleneck')
    did_hit_error = True
  if did_hit_error:
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor)
    with open(bottleneck_path, 'r') as bottleneck_file:
      bottleneck_string = bottleneck_file.read()
    # Allow exceptions to propagate here, since they shouldn't happen after a
    # fresh creation
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  return bottleneck_values

In [56]:
def cache_bottlenecks(sess, image_lists, image_dir, bottleneck_dir,
                      jpeg_data_tensor, decoded_image_tensor,
                      resized_input_tensor, bottleneck_tensor, architecture):
  """Ensures all the training, testing, and validation bottlenecks are cached.

  Because we're likely to read the same image multiple times (if there are no
  distortions applied during training) it can speed things up a lot if we
  calculate the bottleneck layer values once for each image during
  preprocessing, and then just read those cached values repeatedly during
  training. Here we go through all the images we've found, calculate those
  values, and save them off.

  Args:
    sess: The current active TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    image_dir: Root folder string of the subfolders containing the training
    images.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    jpeg_data_tensor: Input tensor for jpeg data from file.
    decoded_image_tensor: The output of decoding and resizing the image.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The penultimate output layer of the graph.
    architecture: The name of the model architecture.

  Returns:
    Nothing.
  """
  how_many_bottlenecks = 0
  ensure_dir_exists(bottleneck_dir)
  for label_name, label_lists in image_lists.items():
    for category in ['training', 'testing', 'validation']:
      category_list = label_lists[category]
      for index, unused_base_name in enumerate(category_list):
        get_or_create_bottleneck(
            sess, image_lists, label_name, index, image_dir, category,
            bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
            resized_input_tensor, bottleneck_tensor, architecture)

        how_many_bottlenecks += 1
        if how_many_bottlenecks % 100 == 0:
          tf.logging.info(
              str(how_many_bottlenecks) + ' bottleneck files created.')

In [57]:
def get_random_cached_bottlenecks(sess, image_lists, how_many, category,
                                  bottleneck_dir, image_dir, jpeg_data_tensor,
                                  decoded_image_tensor, resized_input_tensor,
                                  bottleneck_tensor, architecture):
  """Retrieves bottleneck values for cached images.

  If no distortions are being applied, this function can retrieve the cached
  bottleneck values directly from disk for images. It picks a random set of
  images from the specified category.

  Args:
    sess: Current TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    how_many: If positive, a random sample of this size will be chosen.
    If negative, all bottlenecks will be retrieved.
    category: Name string of which set to pull from - training, testing, or
    validation.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    image_dir: Root folder string of the subfolders containing the training
    images.
    jpeg_data_tensor: The layer to feed jpeg image data into.
    decoded_image_tensor: The output of decoding and resizing the image.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The bottleneck output layer of the CNN graph.
    architecture: The name of the model architecture.

  Returns:
    List of bottleneck arrays, their corresponding ground truths, and the
    relevant filenames.
  """
  class_count = len(image_lists.keys())
  bottlenecks = []
  ground_truths = []
  filenames = []
  if how_many >= 0:
    # Retrieve a random sample of bottlenecks.
    for unused_i in range(how_many):
      label_index = random.randrange(class_count)
      label_name = list(image_lists.keys())[label_index]
      image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
      image_name = get_image_path(image_lists, label_name, image_index,
                                  image_dir, category)
      bottleneck = get_or_create_bottleneck(
          sess, image_lists, label_name, image_index, image_dir, category,
          bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
          resized_input_tensor, bottleneck_tensor, architecture)
      ground_truth = np.zeros(class_count, dtype=np.float32)
      ground_truth[label_index] = 1.0
      bottlenecks.append(bottleneck)
      ground_truths.append(ground_truth)
      filenames.append(image_name)
  else:
    # Retrieve all bottlenecks.
    for label_index, label_name in enumerate(image_lists.keys()):
      for image_index, image_name in enumerate(
          image_lists[label_name][category]):
        image_name = get_image_path(image_lists, label_name, image_index,
                                    image_dir, category)
        bottleneck = get_or_create_bottleneck(
            sess, image_lists, label_name, image_index, image_dir, category,
            bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
            resized_input_tensor, bottleneck_tensor, architecture)
        ground_truth = np.zeros(class_count, dtype=np.float32)
        ground_truth[label_index] = 1.0
        bottlenecks.append(bottleneck)
        ground_truths.append(ground_truth)
        filenames.append(image_name)
  return bottlenecks, ground_truths, filenames

In [58]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [59]:
def add_final_training_ops(class_count, final_tensor_name, bottleneck_tensor,
                           bottleneck_tensor_size):
  """Adds a new softmax and fully-connected layer for training.

  We need to retrain the top layer to identify our new classes, so this function
  adds the right operations to the graph, along with some variables to hold the
  weights, and then sets up all the gradients for the backward pass.

  The set up for the softmax and fully-connected layers is based on:
  https://www.tensorflow.org/versions/master/tutorials/mnist/beginners/index.html

  Args:
    class_count: Integer of how many categories of things we're trying to
    recognize.
    final_tensor_name: Name string for the new final node that produces results.
    bottleneck_tensor: The output of the main CNN graph.
    bottleneck_tensor_size: How many entries in the bottleneck vector.

  Returns:
    The tensors for the training and cross entropy results, and tensors for the
    bottleneck input and ground truth input.
  """
  with tf.name_scope('input'):
    bottleneck_input = tf.placeholder_with_default(
        bottleneck_tensor,
        shape=[None, bottleneck_tensor_size],
        name='BottleneckInputPlaceholder')

    ground_truth_input = tf.placeholder(tf.float32,
                                        [None, class_count],
                                        name='GroundTruthInput')

  # Organizing the following ops as `final_training_ops` so they're easier
  # to see in TensorBoard
  layer_name = 'final_training_ops'
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      initial_value = tf.truncated_normal(
          [bottleneck_tensor_size, class_count], stddev=0.001)

      layer_weights = tf.Variable(initial_value, name='final_weights')

      variable_summaries(layer_weights)
    with tf.name_scope('biases'):
      layer_biases = tf.Variable(tf.zeros([class_count]), name='final_biases')
      variable_summaries(layer_biases)
    with tf.name_scope('Wx_plus_b'):
      logits = tf.matmul(bottleneck_input, layer_weights) + layer_biases
      tf.summary.histogram('pre_activations', logits)

  final_tensor = tf.nn.softmax(logits, name=final_tensor_name)
  tf.summary.histogram('activations', final_tensor)

  with tf.name_scope('cross_entropy'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=ground_truth_input, logits=logits)
    with tf.name_scope('total'):
      cross_entropy_mean = tf.reduce_mean(cross_entropy)
  tf.summary.scalar('cross_entropy', cross_entropy_mean)

  with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_step = optimizer.minimize(cross_entropy_mean)

  return (train_step, cross_entropy_mean, bottleneck_input, ground_truth_input,
          final_tensor)

In [60]:
def add_evaluation_step(result_tensor, ground_truth_tensor):
  """Inserts the operations we need to evaluate the accuracy of our results.

  Args:
    result_tensor: The new final node that produces results.
    ground_truth_tensor: The node we feed ground truth data
    into.

  Returns:
    Tuple of (evaluation step, prediction).
  """
  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      prediction = tf.argmax(result_tensor, 1)
      correct_prediction = tf.equal(
          prediction, tf.argmax(ground_truth_tensor, 1))
    with tf.name_scope('accuracy'):
      evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', evaluation_step)
  return evaluation_step, prediction

In [61]:
def save_graph_to_file(sess, graph, graph_file_name):
  output_graph_def = graph_util.convert_variables_to_constants(
      sess, graph.as_graph_def(), [final_tensor_name])
  with gfile.FastGFile(graph_file_name, 'wb') as f:
    f.write(output_graph_def.SerializeToString())
  return

In [62]:
def prepare_file_system():
  # Setup the directory we'll write summaries to for TensorBoard
  if tf.gfile.Exists(summaries_dir):
    tf.gfile.DeleteRecursively(summaries_dir)
  tf.gfile.MakeDirs(summaries_dir)
  if intermediate_store_frequency > 0:
    ensure_dir_exists(intermediate_output_graphs_dir)
  return

In [63]:
def add_jpeg_decoding(input_width, input_height, input_depth, input_mean,
                      input_std):
  """Adds operations that perform JPEG decoding and resizing to the graph..

  Args:
    input_width: Desired width of the image fed into the recognizer graph.
    input_height: Desired width of the image fed into the recognizer graph.
    input_depth: Desired channels of the image fed into the recognizer graph.
    input_mean: Pixel value that should be zero in the image for the graph.
    input_std: How much to divide the pixel values by before recognition.

  Returns:
    Tensors for the node to feed JPEG data into, and the output of the
      preprocessing steps.
  """
  jpeg_data = tf.placeholder(tf.string, name='DecodeJPGInput')
  decoded_image = tf.image.decode_jpeg(jpeg_data, channels=input_depth)
  decoded_image_as_float = tf.cast(decoded_image, dtype=tf.float32)
  decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
  resize_shape = tf.stack([input_height, input_width])
  resize_shape_as_int = tf.cast(resize_shape, dtype=tf.int32)
  resized_image = tf.image.resize_bilinear(decoded_image_4d,
                                           resize_shape_as_int)
  offset_image = tf.subtract(resized_image, input_mean)
  mul_image = tf.multiply(offset_image, 1.0 / input_std)
  return jpeg_data, mul_image

In [72]:
  # Needed to make sure the logging output is visible.
  # See https://github.com/tensorflow/tensorflow/issues/3047
  tf.logging.set_verbosity(tf.logging.INFO)


  # Set up the pre-trained graph.
  #maybe_download_and_extract(model_info['data_url'])
  graph, bottleneck_tensor, resized_image_tensor = (
      create_model_graph(model_info))

  # Look at the folder structure, and create lists of all the images.
  image_lists = create_image_lists(image_dir, testing_percentage,
                                   validation_percentage)
  class_count = len(image_lists.keys())

  with tf.Session(graph=graph) as sess:
    # Set up the image decoding sub-graph.
    jpeg_data_tensor, decoded_image_tensor = add_jpeg_decoding(
        model_info['input_width'], model_info['input_height'],
        model_info['input_depth'], model_info['input_mean'],
        model_info['input_std'])

      # We'll make sure we've calculated the 'bottleneck' image summaries and
      # cached them on disk.
    cache_bottlenecks(sess, image_lists, image_dir,
                        bottleneck_dir, jpeg_data_tensor,
                        decoded_image_tensor, resized_image_tensor,
                        bottleneck_tensor, architecture)

    # Add the new layer that we'll be training.
    (train_step, cross_entropy, bottleneck_input, ground_truth_input,
     final_tensor) = add_final_training_ops(
         len(image_lists.keys()), final_tensor_name, bottleneck_tensor,
         model_info['bottleneck_tensor_size'])

    # Create the operations we need to evaluate the accuracy of our new layer.
    evaluation_step, prediction = add_evaluation_step(
        final_tensor, ground_truth_input)

    # Merge all the summaries and write them out to the summaries_dir
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(summaries_dir + '/train',
                                         sess.graph)

    validation_writer = tf.summary.FileWriter(
        summaries_dir + '/validation')

    # Set up all our weights to their initial default values.
    init = tf.global_variables_initializer()
    sess.run(init)

    # Run the training for as many cycles
    for i in range(how_many_training_steps):
      # Get a batch of input bottleneck values, either calculated fresh every
      # time with distortions applied, or from the cache stored on disk.
      
      (train_bottlenecks,train_ground_truth, _) = get_random_cached_bottlenecks(
             sess, image_lists, train_batch_size, 'training',
             bottleneck_dir, image_dir, jpeg_data_tensor,
             decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
             architecture)
      # Feed the bottlenecks and ground truth into the graph, and run a training
      # step. Capture training summaries for TensorBoard with the `merged` op.
      train_summary, _ = sess.run(
          [merged, train_step],
          feed_dict={bottleneck_input: train_bottlenecks,
                     ground_truth_input: train_ground_truth})
      train_writer.add_summary(train_summary, i)

      # Every so often, print out how well the graph is training.
      is_last_step = (i + 1 == how_many_training_steps)
      if (i % eval_step_interval) == 0 or is_last_step:
        train_accuracy, cross_entropy_value = sess.run(
            [evaluation_step, cross_entropy],
            feed_dict={bottleneck_input: train_bottlenecks,
                       ground_truth_input: train_ground_truth})
        tf.logging.info('%s: Step %d: Train accuracy = %.1f%%' %
                        (datetime.now(), i, train_accuracy * 100))
        tf.logging.info('%s: Step %d: Cross entropy = %f' %
                        (datetime.now(), i, cross_entropy_value))
        validation_bottlenecks, validation_ground_truth, _ = (
            get_random_cached_bottlenecks(
                sess, image_lists, validation_batch_size, 'validation',
                bottleneck_dir, image_dir, jpeg_data_tensor,
                decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
                architecture))
        # Run a validation step and capture training summaries for TensorBoard
        # with the `merged` op.
        validation_summary, validation_accuracy = sess.run(
            [merged, evaluation_step],
            feed_dict={bottleneck_input: validation_bottlenecks,
                       ground_truth_input: validation_ground_truth})
        validation_writer.add_summary(validation_summary, i)
        tf.logging.info('%s: Step %d: Validation accuracy = %.1f%% (N=%d)' %
                        (datetime.now(), i, validation_accuracy * 100,
                         len(validation_bottlenecks)))

      # Store intermediate results
      intermediate_frequency = intermediate_store_frequency

      if (intermediate_frequency > 0 and (i % intermediate_frequency == 0)
          and i > 0):
        intermediate_file_name = (intermediate_output_graphs_dir +
                                  'intermediate_' + str(i) + '.pb')
        tf.logging.info('Save intermediate result to : ' +
                        intermediate_file_name)
        save_graph_to_file(sess, graph, intermediate_file_name)

    # We've completed all our training, so run a final test evaluation on
    # some new images we haven't used before.
    test_bottlenecks, test_ground_truth, test_filenames = (
        get_random_cached_bottlenecks(
            sess, image_lists, test_batch_size, 'testing',
            bottleneck_dir, image_dir, jpeg_data_tensor,
            decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
            architecture))
    test_accuracy, predictions = sess.run(
        [evaluation_step, prediction],
        feed_dict={bottleneck_input: test_bottlenecks,
                   ground_truth_input: test_ground_truth})
    tf.logging.info('Final test accuracy = %.1f%% (N=%d)' %
                    (test_accuracy * 100, len(test_bottlenecks)))

    if print_misclassified_test_images:
      tf.logging.info('=== MISCLASSIFIED TEST IMAGES ===')
      for i, test_filename in enumerate(test_filenames):
        if predictions[i] != test_ground_truth[i].argmax():
          tf.logging.info('%70s  %s' %
                          (test_filename,
                           list(image_lists.keys())[predictions[i]]))

    # Write out the trained graph and labels with the weights stored as
    # constants.
    save_graph_to_file(sess, graph, output_graph)
    with gfile.FastGFile(output_labels, 'w') as f:
      f.write('\n'.join(image_lists.keys()) + '\n')

INFO:tensorflow:Looking for images in 'Defect'
INFO:tensorflow:Looking for images in 'Healthy'
INFO:tensorflow:100 bottleneck files created.
INFO:tensorflow:200 bottleneck files created.
INFO:tensorflow:300 bottleneck files created.
INFO:tensorflow:400 bottleneck files created.
INFO:tensorflow:500 bottleneck files created.
INFO:tensorflow:2019-05-05 20:03:00.478896: Step 0: Train accuracy = 58.0%
INFO:tensorflow:2019-05-05 20:03:00.494521: Step 0: Cross entropy = 0.678581
INFO:tensorflow:2019-05-05 20:03:01.620685: Step 0: Validation accuracy = 40.0% (N=50)
INFO:tensorflow:2019-05-05 20:03:07.000356: Step 10: Train accuracy = 56.0%
INFO:tensorflow:2019-05-05 20:03:07.000356: Step 10: Cross entropy = 0.832832
INFO:tensorflow:2019-05-05 20:03:07.195211: Step 10: Validation accuracy = 44.0% (N=50)
INFO:tensorflow:2019-05-05 20:03:08.064835: Step 20: Train accuracy = 74.0%
INFO:tensorflow:2019-05-05 20:03:08.064835: Step 20: Cross entropy = 0.581219
INFO:tensorflow:2019-05-05 20:03:08.2043

INFO:tensorflow:2019-05-05 20:03:30.710177: Step 320: Cross entropy = 0.238314
INFO:tensorflow:2019-05-05 20:03:30.790450: Step 320: Validation accuracy = 72.0% (N=50)
INFO:tensorflow:2019-05-05 20:03:31.300318: Step 330: Train accuracy = 94.0%
INFO:tensorflow:2019-05-05 20:03:31.300318: Step 330: Cross entropy = 0.276381
INFO:tensorflow:2019-05-05 20:03:31.366341: Step 330: Validation accuracy = 80.0% (N=50)
INFO:tensorflow:2019-05-05 20:03:31.878008: Step 340: Train accuracy = 98.0%
INFO:tensorflow:2019-05-05 20:03:31.878008: Step 340: Cross entropy = 0.248019
INFO:tensorflow:2019-05-05 20:03:31.929733: Step 340: Validation accuracy = 88.0% (N=50)
INFO:tensorflow:2019-05-05 20:03:32.486277: Step 350: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:03:32.486277: Step 350: Cross entropy = 0.201009
INFO:tensorflow:2019-05-05 20:03:32.546116: Step 350: Validation accuracy = 72.0% (N=50)
INFO:tensorflow:2019-05-05 20:03:33.158594: Step 360: Train accuracy = 98.0%
INFO:tensorflow:201

INFO:tensorflow:2019-05-05 20:03:53.221593: Step 660: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:03:53.221593: Step 660: Cross entropy = 0.125301
INFO:tensorflow:2019-05-05 20:03:53.351671: Step 660: Validation accuracy = 88.0% (N=50)
INFO:tensorflow:2019-05-05 20:03:53.965082: Step 670: Train accuracy = 96.0%
INFO:tensorflow:2019-05-05 20:03:53.965082: Step 670: Cross entropy = 0.177685
INFO:tensorflow:2019-05-05 20:03:54.012997: Step 670: Validation accuracy = 86.0% (N=50)
INFO:tensorflow:2019-05-05 20:03:54.476338: Step 680: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:03:54.476338: Step 680: Cross entropy = 0.177710
INFO:tensorflow:2019-05-05 20:03:54.553616: Step 680: Validation accuracy = 74.0% (N=50)
INFO:tensorflow:2019-05-05 20:03:55.003847: Step 690: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:03:55.003847: Step 690: Cross entropy = 0.160628
INFO:tensorflow:2019-05-05 20:03:55.052966: Step 690: Validation accuracy = 72.0% (N=50)
INFO:tensorflow:2

INFO:tensorflow:2019-05-05 20:04:13.352777: Step 990: Validation accuracy = 80.0% (N=50)
INFO:tensorflow:2019-05-05 20:04:13.869948: Step 1000: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:04:13.869948: Step 1000: Cross entropy = 0.124866
INFO:tensorflow:2019-05-05 20:04:13.947114: Step 1000: Validation accuracy = 64.0% (N=50)
INFO:tensorflow:2019-05-05 20:04:15.425533: Step 1010: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:04:15.425533: Step 1010: Cross entropy = 0.116467
INFO:tensorflow:2019-05-05 20:04:15.537614: Step 1010: Validation accuracy = 82.0% (N=50)
INFO:tensorflow:2019-05-05 20:04:16.346887: Step 1020: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:04:16.362509: Step 1020: Cross entropy = 0.117152
INFO:tensorflow:2019-05-05 20:04:16.429252: Step 1020: Validation accuracy = 88.0% (N=50)
INFO:tensorflow:2019-05-05 20:04:17.220995: Step 1030: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:04:17.220995: Step 1030: Cross entropy = 0.123712
INFO:

INFO:tensorflow:2019-05-05 20:04:43.912882: Step 1320: Validation accuracy = 78.0% (N=50)
INFO:tensorflow:2019-05-05 20:04:44.359952: Step 1330: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:04:44.359952: Step 1330: Cross entropy = 0.101230
INFO:tensorflow:2019-05-05 20:04:44.410922: Step 1330: Validation accuracy = 92.0% (N=50)
INFO:tensorflow:2019-05-05 20:04:44.983333: Step 1340: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:04:44.983333: Step 1340: Cross entropy = 0.110215
INFO:tensorflow:2019-05-05 20:04:45.068709: Step 1340: Validation accuracy = 78.0% (N=50)
INFO:tensorflow:2019-05-05 20:04:45.802739: Step 1350: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:04:45.802739: Step 1350: Cross entropy = 0.067756
INFO:tensorflow:2019-05-05 20:04:45.872407: Step 1350: Validation accuracy = 74.0% (N=50)
INFO:tensorflow:2019-05-05 20:04:46.450704: Step 1360: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:04:46.450704: Step 1360: Cross entropy = 0.099168
INFO

...

INFO:tensorflow:2019-05-05 20:07:30.592253: Step 3630: Validation accuracy = 76.0% (N=50)
INFO:tensorflow:2019-05-05 20:07:31.042811: Step 3640: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:07:31.042811: Step 3640: Cross entropy = 0.031807
INFO:tensorflow:2019-05-05 20:07:31.098477: Step 3640: Validation accuracy = 68.0% (N=50)
INFO:tensorflow:2019-05-05 20:07:31.579968: Step 3650: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:07:31.579968: Step 3650: Cross entropy = 0.038651
INFO:tensorflow:2019-05-05 20:07:31.625676: Step 3650: Validation accuracy = 74.0% (N=50)
INFO:tensorflow:2019-05-05 20:07:32.109153: Step 3660: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:07:32.109153: Step 3660: Cross entropy = 0.033053
INFO:tensorflow:2019-05-05 20:07:32.162928: Step 3660: Validation accuracy = 82.0% (N=50)
INFO:tensorflow:2019-05-05 20:07:32.630022: Step 3670: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:07:32.630022: Step 3670: Cross entropy = 0.050414
INFO

INFO:tensorflow:2019-05-05 20:07:50.832461: Step 3960: Validation accuracy = 80.0% (N=50)
INFO:tensorflow:2019-05-05 20:07:52.326729: Step 3970: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:07:52.326729: Step 3970: Cross entropy = 0.035144
INFO:tensorflow:2019-05-05 20:07:52.476970: Step 3970: Validation accuracy = 72.0% (N=50)
INFO:tensorflow:2019-05-05 20:07:53.362965: Step 3980: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:07:53.362965: Step 3980: Cross entropy = 0.039194
INFO:tensorflow:2019-05-05 20:07:53.457460: Step 3980: Validation accuracy = 86.0% (N=50)
INFO:tensorflow:2019-05-05 20:07:54.007057: Step 3990: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:07:54.007057: Step 3990: Cross entropy = 0.047485
INFO:tensorflow:2019-05-05 20:07:54.079505: Step 3990: Validation accuracy = 70.0% (N=50)
INFO:tensorflow:2019-05-05 20:07:55.261023: Step 4000: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:07:55.261023: Step 4000: Cross entropy = 0.028551
INFO

INFO:tensorflow:2019-05-05 20:08:15.176095: Step 4290: Validation accuracy = 74.0% (N=50)
INFO:tensorflow:2019-05-05 20:08:15.641707: Step 4300: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:08:15.657328: Step 4300: Cross entropy = 0.040831
INFO:tensorflow:2019-05-05 20:08:15.698060: Step 4300: Validation accuracy = 72.0% (N=50)
INFO:tensorflow:2019-05-05 20:08:16.168071: Step 4310: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:08:16.168071: Step 4310: Cross entropy = 0.033115
INFO:tensorflow:2019-05-05 20:08:16.225833: Step 4310: Validation accuracy = 82.0% (N=50)
INFO:tensorflow:2019-05-05 20:08:16.704651: Step 4320: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:08:16.704651: Step 4320: Cross entropy = 0.029178
INFO:tensorflow:2019-05-05 20:08:16.773756: Step 4320: Validation accuracy = 74.0% (N=50)
INFO:tensorflow:2019-05-05 20:08:17.246472: Step 4330: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:08:17.246472: Step 4330: Cross entropy = 0.046862
INFO

INFO:tensorflow:2019-05-05 20:08:44.459973: Step 4620: Validation accuracy = 80.0% (N=50)
INFO:tensorflow:2019-05-05 20:08:44.943646: Step 4630: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:08:44.943646: Step 4630: Cross entropy = 0.038117
INFO:tensorflow:2019-05-05 20:08:44.991897: Step 4630: Validation accuracy = 80.0% (N=50)
INFO:tensorflow:2019-05-05 20:08:45.512592: Step 4640: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:08:45.512592: Step 4640: Cross entropy = 0.026053
INFO:tensorflow:2019-05-05 20:08:45.578084: Step 4640: Validation accuracy = 86.0% (N=50)
INFO:tensorflow:2019-05-05 20:08:46.034371: Step 4650: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:08:46.034371: Step 4650: Cross entropy = 0.034368
INFO:tensorflow:2019-05-05 20:08:46.086799: Step 4650: Validation accuracy = 74.0% (N=50)
INFO:tensorflow:2019-05-05 20:08:46.548776: Step 4660: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:08:46.548776: Step 4660: Cross entropy = 0.039511
INFO

INFO:tensorflow:2019-05-05 20:09:03.608131: Step 4950: Validation accuracy = 88.0% (N=50)
INFO:tensorflow:2019-05-05 20:09:04.077671: Step 4960: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:09:04.077671: Step 4960: Cross entropy = 0.033874
INFO:tensorflow:2019-05-05 20:09:04.143772: Step 4960: Validation accuracy = 80.0% (N=50)
INFO:tensorflow:2019-05-05 20:09:04.594261: Step 4970: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:09:04.594261: Step 4970: Cross entropy = 0.034566
INFO:tensorflow:2019-05-05 20:09:04.664776: Step 4970: Validation accuracy = 88.0% (N=50)
INFO:tensorflow:2019-05-05 20:09:05.358230: Step 4980: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:09:05.358230: Step 4980: Cross entropy = 0.034049
INFO:tensorflow:2019-05-05 20:09:05.424683: Step 4980: Validation accuracy = 80.0% (N=50)
INFO:tensorflow:2019-05-05 20:09:05.908013: Step 4990: Train accuracy = 100.0%
INFO:tensorflow:2019-05-05 20:09:05.908013: Step 4990: Cross entropy = 0.031943
INFO

In [73]:
def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph

In [74]:
def read_tensor_from_image_file(file_name, input_height=299, input_width=299, input_mean=0, input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(file_reader, channels = 3,
                                       name='png_reader')
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(tf.image.decode_gif(file_reader,
                                                  name='gif_reader'))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
  else:
    image_reader = tf.image.decode_jpeg(file_reader, channels = 3,
                                        name='jpeg_reader')
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0);
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)

  return result

In [75]:
def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

In [76]:
file_name = "tf_files/YE358311_Fender_apron/Healthy/IMG20180905151016.jpg"
model_file = "tf_files/retrained_graph.pb"
label_file = "tf_files/retrained_labels.txt"
input_height = 224
input_width = 224
input_mean = 128
input_std = 128
input_layer = "input"
output_layer = "final_result"

In [77]:
  graph = load_graph(model_file)  
  t = read_tensor_from_image_file(file_name,
                                  input_height=input_height,
                                  input_width=input_width,
                                  input_mean=input_mean,
                                  input_std=input_std)

  input_name = "import/" + input_layer
  output_name = "import/" + output_layer
  input_operation = graph.get_operation_by_name(input_name);
  output_operation = graph.get_operation_by_name(output_name);

  with tf.Session(graph=graph) as sess:
    start = time.time()
    results = sess.run(output_operation.outputs[0],
                      {input_operation.outputs[0]: t})
    end=time.time()
  results = np.squeeze(results)

  top_k = results.argsort()[-5:][::-1]
  labels = load_labels(label_file)

  print('\nEvaluation time (1-image): {:.3f}s\n'.format(end-start))
  template = "{} (score={:0.5f})"
  for i in top_k:
    print(template.format(labels[i], results[i]))


Evaluation time (1-image): 3.947s

healthy (score=0.87629)
defect (score=0.12371)
